# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

In [ ]:
## Personal Details:

In [ ]:
# Details Student 1:
# Shay Tekel : 318508405
# Details Student 2:
# Amit Zur : 322280264

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [1]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [2]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [3]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [14]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [10]:
from sklearn.metrics import make_scorer, f1_score # make scorer import

In [11]:
# changing special chars , notes and numbers

def regOnDate(df):
    processed_df = df.copy() # create a new DataFrame with the same shape as the original df
    for i in processed_df.index:
        processed_df.iloc[i,0] = re.sub(r'\W',' ',str(processed_df.iloc[i,0])) 
        processed_df.iloc[i,0] = re.sub(r'\d',' ',str(processed_df.iloc[i,0])) 
        processed_df.iloc[i,0] = re.sub(r'\s\w\s',' ',processed_df.iloc[i,0]) 
    return processed_df['story']

In [12]:
# cleaning the train data 

def my_cleaning():
    df_train['story'] = df_train['story'].apply(lambda x: x.lower())
    df_test['story'] = df_test['story'].apply(lambda x: x.lower())
    df_train['story'] = regOnDate(df_train)
    df_test['story'] = regOnDate(df_test)
    X_train = df_train['story']  # Select only the 'story' column
    y_train = df_train['gender']  # Select only the 'gender' column
    x_test = df_test['story']
    return X_train, y_train , x_test

In [ ]:
# usage of the hebrew tokenizer

def tokenizer_vect( hebrew_text):
    token = ht.tokenize(hebrew_text)
    tokens = list( filter( lambda x : ( x[0] == 'HEBREW') , token))
    tokens = [tuple[1] for tuple in tokens]
    return tokens

In [13]:

x_train , y_train , x_test = my_cleaning()
best_score = -1
good_clf = None

y_pred_saver = None
max_average_score = -1
best_model_saver = None


### Finding Best MODEL:

In the following cells , we will try several models , and compare them in order to find the best model

We will use a variety of vectorizers , models - and different values for each model

###### We will calculate the avg f1 score with the macro f1 , like the assignment's initial requierments  

## KNeighborsClassifier :

In [23]:

# Create a pipeline with KNeighborsClassifier
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 6))),
    ('clf', KNeighborsClassifier())
])

# Define the parameter grid for grid search
param_grid = {
    'vect__ngram_range': [(1, 1), (1, 2)],  # n-gram range for the vectorizer
    'clf__n_neighbors': [3, 5, 7],  # number of neighbors
    'clf__weights': ['uniform', 'distance'],  # weight function used in prediction
    'clf__p': [1, 2]  # distance metric used for the tree
}


# Perform grid search with cross-validation using the pipeline
grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring=make_scorer(f1_score, average='macro'), n_jobs = -1)
grid_search.fit(x_train, y_train)

# Get the best pipeline model from grid search
best_model = grid_search.best_estimator_

# Calculate F1-score using cross-validation on the best model
f1_scores = cross_val_score(best_model, x_train, y_train, scoring=make_scorer(f1_score, average='macro'), cv=10, n_jobs = -1)
f1_average = f1_scores.mean()

print("Best model:", best_model)
print("F1-average score:", f1_average)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(ngram_range=(1, 6))),
                                       ('clf', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'clf__n_neighbors': [3, 5, 7], 'clf__p': [1, 2],
                         'clf__weights': ['uniform', 'distance'],
                         'vect__ngram_range': [(1, 1), (1, 2)]},
             scoring=make_scorer(f1_score, average=macro))

Best model: Pipeline(steps=[('vect', CountVectorizer()),
                ('clf',
                 KNeighborsClassifier(n_neighbors=3, weights='distance'))])
F1-average score: 0.5441303518258724


##### Very low score at 0.54

## DecisionTreeClassifier :

In [42]:
param_grid = {
    'clf__criterion': ['gini', 'entropy'],  # Criterion for splitting
    'clf__max_depth': [None, 5, 10],  # Maximum depth of the tree
    'clf__min_samples_split': [2, 5, 10]  # Minimum number of samples required to split
}



pipe  = Pipeline([
    ('vect', TfidfVectorizer(min_df=5,ngram_range=(1, 6) )),
    ('clf', DecisionTreeClassifier(random_state=42))
])


# Perform grid search with cross-validation using the pipeline
grid_search = GridSearchCV(pipe, param_grid, cv=10, scoring=make_scorer(f1_score, average='macro'), n_jobs = -1)
grid_search.fit(x_train, y_train)

# Get the best pipeline model from grid search
best_model = grid_search.best_estimator_
# Calculate F1-score using cross-validation on the best model
f1_scores = cross_val_score(best_model, x_train, y_train, scoring=make_scorer(f1_score, average='macro'), cv=10, n_jobs = -1)
f1_average = f1_scores.mean()

print("Best model:", best_model)
print("F1-average score:", f1_average)
y_pred = best_model.predict(x_test)




if f1_average > max_average_score:
    max_average_score = f1_average
    best_model_saver = best_model
    y_pred_saver = y_pred

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(min_df=5,
                                                        ngram_range=(1, 6))),
                                       ('clf',
                                        DecisionTreeClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'clf__criterion': ['gini', 'entropy'],
                         'clf__max_depth': [None, 5, 10],
                         'clf__min_samples_split': [2, 5, 10]},
             scoring=make_scorer(f1_score, average=macro))

Best model: Pipeline(steps=[('vect', TfidfVectorizer(min_df=5, ngram_range=(1, 6))),
                ('clf',
                 DecisionTreeClassifier(criterion='entropy',
                                        min_samples_split=5,
                                        random_state=42))])
F1-average score: 0.6241829745087971


## Perceptron :

In [41]:
# this time we will use the hebrew 

pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenizer_vect)),
    ('classifier', Perceptron())
])

# Defining the hyperparameters to tune

param_grid = {
    'classifier__penalty': ['l1', 'l2', 'elasticnet'],
    'classifier__alpha': [0.0001, 0.001, 0.01],
    'classifier__max_iter': [1000, 5000, 10000]
}


# Perform grid search with cross-validation using the pipeline
grid_search = GridSearchCV(pipe, param_grid, cv=10, scoring=make_scorer(f1_score, average='macro'), n_jobs = -1)
grid_search.fit(x_train, y_train)

# Get the best pipeline model from grid search
best_model = grid_search.best_estimator_
# Calculate F1-score using cross-validation on the best model
f1_scores = cross_val_score(best_model, x_train, y_train, scoring=make_scorer(f1_score, average='macro'), cv=10, n_jobs = -1)
f1_average = f1_scores.mean()

print("Best model:", best_model)
print("F1-average score:", f1_average)
y_pred = best_model.predict(x_test)




if f1_average > max_average_score:
    max_average_score = f1_average
    best_model_saver = best_model
    y_pred_saver = y_pred

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenizer_vect at 0x0000024C9E371670>)),
                                       ('classifier', Perceptron())]),
             n_jobs=-1,
             param_grid={'classifier__alpha': [0.0001, 0.001, 0.01],
                         'classifier__max_iter': [1000, 5000, 10000],
                         'classifier__penalty': ['l1', 'l2', 'elasticnet']},
             scoring=make_scorer(f1_score, average=macro))

Best model: Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenizer_vect at 0x0000024C9E371670>)),
                ('classifier', Perceptron(penalty='l1'))])
F1-average score: 0.6558884013335995


## SGDClassifier :

In [34]:

params = {
          'vect__min_df': [1, 5],
          'vect__ngram_range': [(1,1), (1,2)],
          'clf__alpha': [ 0.0001, 0.001 , 0.01 , 0.1],
          'clf__penalty': ['l1', 'l2', 'elasticnet'],
          'clf__max_iter': [1000, 2000, 3000],
          'clf__eta0': [0.1, 0.01, 0.001]}

# Defining the pipeline

pipe = Pipeline([
    ('vect', CountVectorizer( tokenizer = tokenizer_vect )),
    ('normalize', preprocessing.Normalizer(norm = 'l2')),
    ('clf', SGDClassifier( max_iter = 1000 , random_state = 42))
])


# Perform grid search with cross-validation using the pipeline
grid_search = GridSearchCV(pipe, params, cv=10, scoring=make_scorer(f1_score, average='macro') , n_jobs = -1)

print("--------------------------------")
grid_search.fit(x_train, y_train )

# Get the best pipeline model from grid search

best_model = grid_search.best_estimator_

# Calculate F1-score using cross-validation on the best model
print("---------------------------------")

f1_scores = cross_val_score(best_model, x_train, y_train, scoring=make_scorer(f1_score, average='macro'), cv=10 , n_jobs = -1)
f1_average = f1_scores.mean()

print("Best model:", best_model)
print("F1-average score:", f1_average)
y_pred = best_model.predict(x_test)




if f1_average > max_average_score:
    max_average_score = f1_average
    best_model_saver = best_model
    y_pred_saver = y_pred

--------------------------------


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenizer_vect at 0x0000024C9E371670>)),
                                       ('normalize', Normalizer()),
                                       ('clf',
                                        SGDClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
                         'clf__eta0': [0.1, 0.01, 0.001],
                         'clf__max_iter': [1000, 2000, 3000],
                         'clf__penalty': ['l1', 'l2', 'elasticnet'],
                         'vect__min_df': [1, 5],
                         'vect__ngram_range': [(1, 1), (1, 2)]},
             scoring=make_scorer(f1_score, average=macro))

---------------------------------
Best model: Pipeline(steps=[('vect',
                 CountVectorizer(min_df=5, ngram_range=(1, 2),
                                 tokenizer=<function tokenizer_vect at 0x0000024C9E371670>)),
                ('normalize', Normalizer()),
                ('clf',
                 SGDClassifier(eta0=0.1, penalty='l1', random_state=42))])
F1-average score: 0.6941253704762893


##### Current best f1 : 0.694

## LinearSVC :

In [35]:
# Defining the pipeline

pipe = Pipeline([
    ('tfidf', TfidfVectorizer( tokenizer = tokenizer_vect  )),
    ('clf', LinearSVC())
])

# Defining the hyperparameters to be tuned:

params = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__max_df': (0.5, 0.75, 1.0),
    'tfidf__min_df': (1, 2),
    'clf__C': (0.1, 1, 10),
    'clf__penalty': ('l1', 'l2'),
    'clf__loss': ('hinge', 'squared_hinge'),
    'clf__dual': (True, False)
}

# Perform grid search with cross-validation using the pipeline

grid_search = GridSearchCV(pipe, params, cv=10, scoring=make_scorer(f1_score, average='macro'), n_jobs = -1)
grid_search.fit(x_train, y_train)

# Get the best pipeline model from grid search

best_model = grid_search.best_estimator_

# Calculate F1-score using cross-validation on the best model

f1_scores = cross_val_score(best_model, x_train, y_train, scoring=make_scorer(f1_score, average='macro'), cv=10, n_jobs = -1)
f1_average = f1_scores.mean()

print("Best model:", best_model)
print("F1-average score:", f1_average)
y_pred = best_model.predict(x_test)




if f1_average > max_average_score:
    max_average_score = f1_average
    best_model_saver = best_model
    y_pred_saver = y_pred

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function tokenizer_vect at 0x0000024C9E371670>)),
                                       ('clf', LinearSVC())]),
             n_jobs=-1,
             param_grid={'clf__C': (0.1, 1, 10), 'clf__dual': (True, False),
                         'clf__loss': ('hinge', 'squared_hinge'),
                         'clf__penalty': ('l1', 'l2'),
                         'tfidf__max_df': (0.5, 0.75, 1.0),
                         'tfidf__min_df': (1, 2),
                         'tfidf__ngram_range': [(1, 1), (1, 2)],
                         'tfidf__use_idf': (True, False)},
             scoring=make_scorer(f1_score, average=macro))

Best model: Pipeline(steps=[('tfidf',
                 TfidfVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function tokenizer_vect at 0x0000024C9E371670>)),
                ('clf', LinearSVC(C=10, dual=False, penalty='l1'))])
F1-average score: 0.7087853043557291


##### current best f1 : 0.708

## MLPClassifier : 

In [16]:
# Defining the pipeline

pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MLPClassifier())
])

# Defining the hyperparameters for a final improved model:

params = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'classifier__hidden_layer_sizes': [(50,), (100,),],
    'classifier__activation': ['logistic', 'relu'],
    'classifier__solver': ['sgd', 'adam'],
    'classifier__max_iter': [1000, 2000]
}


# Perform grid search with cross-validation using the pipeline
grid_search = GridSearchCV(pipe, params, cv=10, scoring=make_scorer(f1_score, average='macro'), n_jobs = -1)
print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')

grid_search.fit(x_train, y_train)

# Get the best pipeline model from grid search

best_model = grid_search.best_estimator_
print('======================')
# Calculate F1-score using cross-validation on the best model

f1_scores = cross_val_score(best_model, x_train, y_train, scoring=make_scorer(f1_score, average='macro'), cv=10, n_jobs = -1)
f1_average = f1_scores.mean()

print("Best model:", best_model)
print("F1-average score:", f1_average)
y_pred = best_model.predict(x_test)




if f1_average > max_average_score:
    max_average_score = f1_average
    best_model_saver = best_model
    y_pred_saver = y_pred

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf', MLPClassifier())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.1, 0.01],
                         'clf__hidden_layer_sizes': [(10,), (20,)]},
             scoring=make_scorer(f1_score, average=macro))

Best model: Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', MLPClassifier(alpha=0.1, hidden_layer_sizes=(20,)))])
F1-average score: 0.5537471930204838


##### We've tried MultinomialNB and MLPclassifier , both return rather low results

In [39]:
best_model_saver
max_average_score

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function tokenizer_vect at 0x0000024C9E371670>)),
                ('clf', LinearSVC(C=10, dual=False, penalty='l1'))])

0.7087853043557291

### Final Result :
#### Best F1 score : 0.708
#### Best Model  : LinearSVC

##### Now , we will predict on the test set :

In [37]:
X_test = df_test['story']
predDfTest = best_model_saver.predict(X_test) # predicting if the author is male or female
test = df_test.copy()

for i in range(0,len(test['test_example_id'])):
    test['test_example_id'][i] = predDfTest[i] #inserting the predictions into the data itself

###### In the tabs bellow , you can see the dataset's 5 first and last rows - of the original 
###### and the predicted data: 

In [38]:
df_predicted = pd.DataFrame(columns = ['test_example_id','predicted_category'])

df_predicted['test_example_id'] = df_test['test_example_id']
df_predicted['predicted_category'] = test['test_example_id']
test
df_predicted

,test_example_id,story
0,m,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,m,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,m,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...
3,m,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני..."
4,f,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...
...,...,...
318,m,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה..."
319,m,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...
320,m,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...
321,m,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ..."


,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [ ]:
df_predicted.to_csv('classification_results.csv',index=False)